In [1]:
import numpy as np
import tensorflow as tf
import compas_data as compas
from sklearn import linear_model
import scipy

In [6]:
x_train, x_test, y_train, y_test, y_sex_train, y_sex_test,\
     y_race_train, y_race_test, feature_names = compas.get_compas_train_test()
x_train, x_test = x_train[:, 2:], x_test[:, 2:]
def standardize(x):
    return (x - np.mean(x))/np.std(x)
for i in range(1, 5):
    x_train[:, i] = standardize(x_train[:, i])
    x_test[:, i] = standardize(x_test[:, i])

(5278, 8)
['sex', 'race', 'priors_count', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
sex
race


In [7]:
sensetive_directions = []
protected_regression = linear_model.LogisticRegression(fit_intercept = True)
protected_regression.fit(x_train, y_sex_train)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
protected_regression.fit(x_train, y_race_train)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
sensetive_directions = np.array(sensetive_directions)

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
    while np.linalg.norm(s) != 1:
        s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s

In [8]:
x_train, x_test = tf.cast(x_train, dtype=tf.float32), tf.cast(x_test, dtype = tf.float32)
sensetive_directions = tf.cast(sensetive_directions, dtype=tf.float32)
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)

In [9]:
x_train

<tf.Tensor: shape=(4222, 5), dtype=float32, numpy=
array([[-0.7103805 , -1.1656154 ,  1.9390371 , -0.5216032 , -1.3554946 ],
       [-0.50492585, -1.1656154 ,  1.9390371 , -0.5216032 , -1.3554946 ],
       [-0.7103805 , -1.1656154 ,  1.9390371 , -0.5216032 ,  0.7377381 ],
       ...,
       [-0.09401665,  0.85791594, -0.5157199 , -0.5216032 ,  0.7377381 ],
       [-0.7103805 , -1.1656154 ,  1.9390371 , -0.5216032 ,  0.7377381 ],
       [-0.7103805 , -1.1656154 , -0.5157199 ,  1.9171661 ,  0.7377381 ]],
      dtype=float32)>

In [10]:
x_test

<tf.Tensor: shape=(1056, 5), dtype=float32, numpy=
array([[-0.50492585,  0.8819171 , -0.49674264, -0.56130964,  0.70409566],
       [-0.7103805 ,  0.8819171 , -0.49674264, -0.56130964,  0.70409566],
       [-0.7103805 ,  0.8819171 , -0.49674264, -0.56130964,  0.70409566],
       ...,
       [-0.29947126, -1.1338934 , -0.49674264,  1.7815479 , -1.4202615 ],
       [ 1.3441656 , -1.1338934 ,  2.013115  , -0.56130964,  0.70409566],
       [ 0.31689256,  0.8819171 , -0.49674264, -0.56130964, -1.4202615 ]],
      dtype=float32)>

In [11]:
sensetive_directions

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-0.8898071 , -0.00434467,  0.30173835, -0.29453692, -0.17443155],
       [ 0.4009065 , -0.2892771 ,  0.76533276, -0.4117101 , -0.01879575]],
      dtype=float32)>